In [11]:
import sys, os
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import statsmodels.api as sm
from scipy import stats
import scipy.stats.distributions as dist

from matplotlib import pyplot as plt
import matplotlib as mpl
#mpl.style.use("dark_background")

In [12]:
def load(ticker: str, cache=True):
    import os
    filename = f"{ticker}.csv"
    if cache and os.path.isfile(filename):
        df = pd.read_csv(filename)
        df.reset_index("date")
        return df
    else:
        df = pdr.get_data_tiingo(ticker, api_key="a1955b6b5ab1cd5bce14053378f1f9358f1e65cb")
        df = df.loc[ticker]
        df.reset_index("date")
        df.to_csv(filename)
        return df

In [13]:
df = load("GOOG", cache=False)
df

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2015-06-17 00:00:00+00:00,529.26,530.980,525.10,529.37,1294216,529.26,530.980,525.10,529.37,1294216,0.0,1.0
2015-06-18 00:00:00+00:00,536.73,538.150,530.79,531.00,1833109,536.73,538.150,530.79,531.00,1833109,0.0,1.0
2015-06-19 00:00:00+00:00,536.69,538.250,533.01,537.21,1893497,536.69,538.250,533.01,537.21,1893497,0.0,1.0
2015-06-22 00:00:00+00:00,538.19,543.740,537.53,539.59,1250282,538.19,543.740,537.53,539.59,1250282,0.0,1.0
2015-06-23 00:00:00+00:00,540.48,541.499,535.25,539.64,1197450,540.48,541.499,535.25,539.64,1197450,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-09 00:00:00+00:00,1456.16,1468.000,1443.21,1445.36,1409249,1456.16,1468.000,1443.21,1445.36,1409249,0.0,1.0
2020-06-10 00:00:00+00:00,1465.85,1474.260,1456.27,1459.54,1525153,1465.85,1474.260,1456.27,1459.54,1525153,0.0,1.0
2020-06-11 00:00:00+00:00,1403.84,1454.470,1402.00,1442.48,1991332,1403.84,1454.470,1402.00,1442.48,1991332,0.0,1.0


In [14]:
%matplotlib widget
closes = df["close"]
closes.plot(figsize=(12,8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# returns = closes.pct_change()[1:]
returns = np.log(closes).diff()[1:]
print(f"Mean {returns.mean():%}, std {returns.std():.2%}, skewness {returns.skew()}, excess kurtosis {returns.kurtosis()}")

Mean 0.078507%, std 1.71%, skewness 0.3043123344680597, excess kurtosis 10.469479201001892


In [16]:
%matplotlib widget
ax = returns.hist(bins=70, density=True, label="Histogram")
approx_norm = dist.norm(loc=returns.mean(), scale=returns.std())
x = np.linspace(returns.min(), returns.max(), 1000)
ax.plot(x, approx_norm.pdf(x), label="Approx. Gaussian")
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
sm.qqplot(returns, fit=True, line="45")
pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
k2, p = stats.normaltest(np.random.gamma(3, size=100))
print(f"p-value p = {p:.2}")
alpha = 0.001
if p < alpha:
    print("Null hypothesis can be rejected: returns are not Gaussian")
else:
    print("Null hypothesis cannot be rejected.: returns may be Gaussian")

p-value p = 0.013
Null hypothesis cannot be rejected.: returns may be Gaussian


In [19]:
%matplotlib widget
window = 300
series = returns.rolling(window).kurt()[window-1:]
plt.plot(series.index, series, label=f"{window}D-rolling excess Kurtosis")
plt.plot(returns.rolling(window).skew()[window-1:], label=f"{window}D-rolling Skewness")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Statistical analysis

In [26]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, 

In [32]:
plot_acf(returns)
plot_pacf(returns)
pass

/Users/edouard/.virtualenvs/stats/lib/python3.7/site-packages/statsmodels/graphics/utils.py:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/edouard/.virtualenvs/stats/lib/python3.7/site-packages/statsmodels/graphics/utils.py:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
from arch import arch_model
from sklearn.preprocessing import StandardScaler

### Train - test split

In [44]:
train_test_split = 0.80
train_split = int(len(returns) * 0.80)
train, test = returns[:train_split], returns[train_split:]

### Scaling

In [68]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train.values.reshape(-1,1))

In [90]:
res.aic

2746.341893644617

In [106]:
import sys

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [141]:
range_o = list(range(1, 6))
range_p = list(range(1, 10))
range_q = list(range(1, 17))
for o in range_o:
    print(f"o = {o}")
    best_aic, best_model = sys.maxsize, None
    aic = np.zeros(shape=(len(range_p), len(range_q)))
    for i, p in enumerate(range_p):
        for j, q in enumerate(range_q):
            model = arch_model(train_scaled, vol="GARCH", p=p, o=o, q=q)
            res = model.fit(disp="off")
            aic[i, j] = res.aic
            if res.aic < best_aic:
                best_aic, best_model = res.aic, (p,q)
    
    X, Y = np.meshgrid(range_p, range_q)
    ax = Axes3D(fig=plt.figure())
    ax.plot_surface(X, Y, aic.T)
    ax.set_title(f"o = {o}")
    plt.show()
    print(f"Best (p, q) = {best_model}, AIC = {best_aic}")

o = 1


/Users/edouard/.virtualenvs/stats/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Best (p, q) = (4, 11), AIC = 2700.4927262261276
o = 2


/Users/edouard/.virtualenvs/stats/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Best (p, q) = (4, 11), AIC = 2691.7527752654064
o = 5


/Users/edouard/.virtualenvs/stats/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Best (p, q) = (4, 11), AIC = 2693.752776110158


### Fitting

In [130]:
p, o, q = 4, 1, 11
model = arch_model(train_scaled, p=p, o=o, q=q)
res = model.fit(disp="off")
pass

### Prediction

In [131]:
test_scaled = scaler.transform(test.values.reshape(-1,1))
predict = res.forecast(horizon=len(test_scaled))

In [140]:
predict.variance

,h.001,h.002,h.003,h.004,h.005,h.006,h.007,h.008,h.009,h.010,...,h.243,h.244,h.245,h.246,h.247,h.248,h.249,h.250,h.251,h.252
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
